In [62]:
pip install -q flwr[simulation]


In [63]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import flwr as fl
import logging
import math
import warnings

# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

logging.basicConfig(format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S", level=logging.INFO)

#Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, explained_variance_score, confusion_matrix, accuracy_score, classification_report, log_loss
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

#Tensorflow para redes neuronales
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Activation
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras import backend as K

from typing import List, Tuple, Dict, Optional
from flwr.common import Metrics
import time

# Cambiamos el display para que nos muestre todas las columnas
pd.set_option("display.max_rows", None)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Cargamos los datasets sampleados

In [4]:
DIRECTORIO = "/content/drive/My Drive/UNIVERSIDAD/TFG/Flower/datasets/"


In [5]:
df_cic_sampled = pd.read_csv(os.path.join(DIRECTORIO, "cicids_sampled.csv"), skipinitialspace=True, error_bad_lines=False )

<ipython-input-5-dd721b120a09>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_cic_sampled = pd.read_csv(os.path.join(DIRECTORIO, "cicids_sampled.csv"), skipinitialspace=True, error_bad_lines=False )


In [6]:
df_nb15_sampled = pd.read_csv(os.path.join(DIRECTORIO, "nb15_sampled.csv"), skipinitialspace=True, error_bad_lines=False )

<ipython-input-6-407920532375>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_nb15_sampled = pd.read_csv(os.path.join(DIRECTORIO, "nb15_sampled.csv"), skipinitialspace=True, error_bad_lines=False )


In [7]:
df_toniot_sampled = pd.read_csv(os.path.join(DIRECTORIO, "toniot_sampled.csv"), skipinitialspace=True, error_bad_lines=False )

<ipython-input-7-cc2ff887f1ab>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_toniot_sampled = pd.read_csv(os.path.join(DIRECTORIO, "toniot_sampled.csv"), skipinitialspace=True, error_bad_lines=False )


Eliminamos el atributo 'IPV4_SRC_ADDR', ya que además de ser un String, un atacante puede modificar sus direcciones ip con facilidad por lo que no es un buen indicador de ataques

# Detección de ataques de forma binaria

In [8]:
df_cic_sampled = df_cic_sampled.drop(columns=['IPV4_SRC_ADDR','Dataset','Attack', 'IPV4_DST_ADDR'])
df_nb15_sampled = df_nb15_sampled.drop(columns=['IPV4_SRC_ADDR','Dataset','Attack', 'IPV4_DST_ADDR'])
df_toniot_sampled = df_toniot_sampled.drop(columns=['IPV4_SRC_ADDR','Dataset','Attack', 'IPV4_DST_ADDR'])

In [9]:
df_cic_sampled.head()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,MIN_TTL,MAX_TTL,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,SRC_TO_DST_SECOND_BYTES,DST_TO_SRC_SECOND_BYTES,RETRANSMITTED_IN_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,RETRANSMITTED_OUT_PKTS,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
0,49171,3389,6,0.000,1460,8,2126,10,222,222,90,0,0,0,98,98,1213,40,40,1213,1460.0,2126.0,0,0,253,3,11680000,17008000,13,2,1,1,1,8192,64000,0,0,0,0,0,0.0,0
1,60992,3389,6,0.000,1830,10,3600,12,30,30,26,0,0,0,103,103,1213,40,40,1213,1830.0,3600.0,370,2,1623,3,14640000,28800000,14,3,2,1,2,8192,64000,29184,114,0,0,0,0.0,0
2,54899,53,17,5.126,71,1,87,1,0,0,0,0,0,0,0,0,87,71,71,87,71.0,87.0,0,0,0,0,568000,696000,2,0,0,0,0,0,0,0,0,45060,1,60,0.0,0
3,51920,80,6,7.126,1136,17,1450,15,27,27,27,0,0,0,128,128,746,40,40,746,1136.0,1450.0,451,11,0,0,9088000,11600000,30,0,1,1,0,8192,42780,49920,195,0,0,0,0.0,0
4,60596,3389,6,0.000,1460,8,1873,7,222,222,90,0,0,0,102,102,1213,40,40,1213,1460.0,1873.0,0,0,0,0,11680000,14984000,10,2,1,1,1,8192,64000,0,0,0,0,0,0.0,0


In [ ]:
# info(), describe()
df_cic_sampled.describe()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,MIN_TTL,MAX_TTL,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,SRC_TO_DST_SECOND_BYTES,DST_TO_SRC_SECOND_BYTES,RETRANSMITTED_IN_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,RETRANSMITTED_OUT_PKTS,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1.000000e+06,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1.000000e+06,1.000000e+06,1000000.000000,1000000.00000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.0,1000000.000000
mean,52659.920038,1284.360772,10.548983,12.403945,1.740838e+03,22.161183,7.421483e+03,10.060070,76.299446,75.594511,21.262760,5.740977e+05,23.260189,11.283958,71.586959,71.695324,564.173963,53.587941,49.403967,564.173963,3.433269e+284,9.549449e+15,38.895994,0.544643,5.034912e+01,0.115312,4.762750e+06,1.400946e+07,25.067955,1.34409,0.742837,0.615475,4.541102,11521.682196,20960.230931,3243.569417,12.670189,13377.853879,3.901842,19.153562,0.0,0.119377
std,11642.282164,3772.600236,5.435479,28.713816,6.437620e+04,1038.055828,5.257403e+05,353.651708,99.070738,98.840040,28.909778,1.461522e+06,1019.594722,74.125931,56.891609,57.004592,540.273537,36.367985,19.578821,540.273537,inf,9.549185e+18,283.660998,2.983208,2.569057e+03,2.165843,1.139812e+07,9.683141e+07,1038.064961,6.03406,4.639909,4.776127,349.821967,19104.998681,25035.197726,12051.384688,47.075722,20137.412899,15.596916,26.367543,0.0,0.324232
min,0.000000,0.000000,1.000000,0.000000,2.800000e+01,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,28.000000,28.000000,0.000000,28.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,50725.000000,53.000000,6.000000,0.000000,6.900000e+01,1.000000,1.170000e+02,1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,100.000000,40.000000,40.000000,100.000000,7.000000e+01,1.210000e+02,0.000000,0.000000,0.000000e+00,0.000000,5.280000e+05,7.600000e+05,2.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,54193.000000,80.000000,6.000000,0.000000,1.800000e+02,4.000000,3.220000e+02,3.000000,27.000000,26.000000,22.000000,0.000000e+00,0.000000,0.000000,101.000000,101.000000,191.000000,40.000000,40.000000,191.000000,2.080000e+02,3.380000e+02,0.000000,0.000000,0.000000e+00,0.000000,9.920000e+05,1.440000e+06,6.000000,1.00000,0.000000,0.000000,0.000000,8192.000000,8192.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,59507.000000,445.000000,17.000000,7.000000,1.460000e+03,9.000000,1.873000e+03,7.000000,219.000000,219.000000,27.000000,0.000000e+00,0.000000,0.000000,128.000000,128.000000,1189.000000,66.000000,65.000000,1189.000000,1.460000e+03,1.873000e+03,0.000000,0.000000,0.000000e+00,0.000000,9.096000e+06,1.376800e+07,12.000000,2.00000,1.000000,1.000000,1.000000,8192.000000,29200.000000,0.000000,0.000000,25392.000000,1.000000,60.000000,0.0,0.000000
max,65535.000000,65535.000000,58.000000,248.000000,1.652742e+07,200

# Preprocesamiento de los datasets

1º Eliminar valores nulos (si existen)

In [10]:
df_cic_sampled = df_cic_sampled.dropna()
df_nb15_sampled = df_nb15_sampled.dropna()
df_toniot_sampled = df_toniot_sampled.dropna()

In [ ]:
corr_matrix = df_cic_sampled.corr()
corr_with_target = corr_matrix['Label']

# imprimir coeficientes de correlación ordenados de mayor a menor
print(corr_with_target.abs().sort_values(ascending=False))

In [ ]:
corr_matrix2 = df_nb15_sampled.corr()
corr_with_target2 = corr_matrix2['Label']

# imprimir coeficientes de correlación ordenados de mayor a menor
print(corr_with_target2.abs().sort_values(ascending=False))

In [ ]:
corr_matrix3 = df_toniot_sampled.corr()
corr_with_target3 = corr_matrix3['Label']

# imprimir coeficientes de correlación ordenados de mayor a menor
print(corr_with_target3.abs().sort_values(ascending=False))

In [ ]:
# Change inf value with maximum value of each class
inf_rows = df_cic_sampled[df_cic_sampled.isin([np.inf]).any(axis=1)].shape[0]
logging.info("Replace Inf in {} rows with maximum value of each class.".format(inf_rows))
# Temporary replace inf with NaN
df_cic_sampled = df_cic_sampled.replace([np.inf], np.nan)
# Replace inf with maximum value of each class in this dataset
df_cic_sampled.iloc[:, df_cic_sampled.columns != "Label"] = df_cic_sampled.groupby("Label").transform(lambda x: x.fillna(x.max()))

# Change negative value with minimum positive value of each class
logging.info("Replace negative values with minimum value of each class.")
# Temporary replace negative value with NaN
df_cic_sampled.iloc[:, :-1][df_cic_sampled.iloc[:, :-1] < 0] = np.nan
# Replace negative value with minimum value of each class in this dataset
df_cic_sampled.iloc[:, df_cic_sampled.columns != "Label"] = df_cic_sampled.groupby("Label").transform(lambda x: x.fillna(x.min()))

In [11]:
indexes = []
benign_included_count = 0
for index, row in df_cic_sampled.iterrows():
    if row['Label'] != 0:
        indexes.append(index)
    else:
        if benign_included_count < 200000:
            indexes.append(index)
            benign_included_count += 1
df_cic_sampled = df_cic_sampled.loc[indexes]

In [12]:
indexes = []
benign_included_count = 0
for index, row in df_nb15_sampled.iterrows():
    if row['Label'] != 0:
        indexes.append(index)
    else:
        if benign_included_count < 200000:
            indexes.append(index)
            benign_included_count += 1
df_nb15_sampled = df_nb15_sampled.loc[indexes]

In [13]:
indexes = []
benign_included_count = 0
for index, row in df_toniot_sampled.iterrows():
    if row['Label'] != 0:
        indexes.append(index)
    else:
        if benign_included_count < 200000:
            indexes.append(index)
            benign_included_count += 1
df_toniot_sampled = df_toniot_sampled.loc[indexes]

In [14]:
feature_names = [
    "FLOW_DURATION_MILLISECONDS",
    "TCP_WIN_MAX_IN",
    "DURATION_OUT",
    "CLIENT_TCP_FLAGS",
    "TCP_FLAGS",
    "PROTOCOL",
    "DNS_TTL_ANSWER",
    "DNS_QUERY_ID",
    "MIN_TTL",
    "MAX_TTL",
    "LONGEST_FLOW_PKT",
    "MAX_IP_PKT_LEN",
    "MIN_IP_PKT_LEN",
    "SHORTEST_FLOW_PKT",
    "SRC_TO_DST_AVG_THROUGHPUT",
    "RETRANSMITTED_IN_BYTES",
    "DNS_QUERY_TYPE",
    "L7_PROTO",
    "NUM_PKTS_512_TO_1024_BYTES",
    "NUM_PKTS_256_TO_512_BYTES",
    "TCP_WIN_MAX_OUT",
    "SERVER_TCP_FLAGS"
]
feature_names2 = [
    "FLOW_DURATION_MILLISECONDS",
    "TCP_WIN_MAX_IN",
    "DURATION_OUT",
    "CLIENT_TCP_FLAGS",
    "TCP_FLAGS",
    "PROTOCOL",
    "DNS_TTL_ANSWER",
    "DNS_QUERY_ID",
    "MIN_TTL",
    "MAX_TTL",
    "LONGEST_FLOW_PKT",
    "MAX_IP_PKT_LEN",
    "MIN_IP_PKT_LEN",
    "SHORTEST_FLOW_PKT",
    "SRC_TO_DST_AVG_THROUGHPUT",
    "RETRANSMITTED_IN_BYTES",
    "DNS_QUERY_TYPE",
    "L7_PROTO",
    "NUM_PKTS_512_TO_1024_BYTES",
    "NUM_PKTS_256_TO_512_BYTES",
    "TCP_WIN_MAX_OUT",
    "SERVER_TCP_FLAGS",
    "L4_SRC_PORT",
    "L4_DST_PORT",
    "DURATION_IN",
    "NUM_PKTS_128_TO_256_BYTES",
    "DST_TO_SRC_AVG_THROUGHPUT",
    "NUM_PKTS_UP_TO_128_BYTES",
    "IN_PKTS",
    "IN_BYTES",
    "RETRANSMITTED_OUT_PKTS",
    "RETRANSMITTED_IN_PKTS",
    "ICMP_TYPE",
    "ICMP_IPV4_TYPE",
    "RETRANSMITTED_OUT_BYTES"
]




# Partición del dataset en entrenamiento y test

In [18]:
# X_cic = df_cic_sampled.drop(['Label'], axis=1)
X_cic = df_cic_sampled[feature_names]
y_cic = df_cic_sampled['Label']

# Split data into train and test
X_train_cic, X_test_cic, y_train_cic, y_test_cic = train_test_split(X_cic, y_cic,shuffle=True, test_size=0.2, random_state=42)

print("X Train shape:",X_train_cic.shape)
print("Y Train shape:",y_train_cic.shape)
print("X Test shape:",X_test_cic.shape)
print("Y Test shape:",y_test_cic.shape)

X Train shape: (255501, 22)
Y Train shape: (255501,)
X Test shape: (63876, 22)
Y Test shape: (63876,)


In [19]:
# X_nb15 = df_nb15_sampled.drop(['Label'], axis=1) # axis=0 se referiria a filas
X_nb15 = df_nb15_sampled[feature_names]
y_nb15 = df_nb15_sampled['Label']

# Split data into train and test
X_train_nb15, X_test_nb15, y_train_nb15, y_test_nb15 = train_test_split(X_nb15, y_nb15,shuffle=True, test_size=0.2, random_state=42)

print("X Train shape:",X_train_nb15.shape)
print("Y Train shape:",y_train_nb15.shape)
print("X Test shape:",X_test_nb15.shape)
print("Y Test shape:",y_test_nb15.shape)

X Train shape: (191937, 22)
Y Train shape: (191937,)
X Test shape: (47985, 22)
Y Test shape: (47985,)


In [20]:
# X_toniot = df_toniot_sampled.drop(['Label'], axis=1)
X_toniot = df_toniot_sampled[feature_names]
y_toniot = df_toniot_sampled['Label']

# Split data into train and test
X_train_toniot, X_test_toniot, y_train_toniot, y_test_toniot = train_test_split(X_toniot, y_toniot,shuffle=True, test_size=0.2, random_state=42)

print("X Train shape:",X_train_toniot.shape)
print("Y Train shape:",y_train_toniot.shape)
print("X Test shape:",X_test_toniot.shape)
print("Y Test shape:",y_test_toniot.shape)

X Train shape: (672305, 22)
Y Train shape: (672305,)
X Test shape: (168077, 22)
Y Test shape: (168077,)


# Modelo Regresión Logística (Centralizado)
Métricas del CIC-ids2017

In [44]:
log_reg = LogisticRegression(random_state=10, solver = 'lbfgs', max_iter=300)
log_reg.fit(X_train_cic, y_train_cic)

LogisticRegression(max_iter=300, random_state=10)

In [45]:
y_pred_test = log_reg.predict(X_test_cic)


In [46]:
# Imprimir métricas de clasificación
from sklearn.metrics import classification_report
print("The Training Accuracy is: ", log_reg.score(X_train_cic, y_train_cic))

# Accuracy on Test
print("The Testing Accuracy is: ", log_reg.score(X_test_cic, y_test_cic))
print(classification_report(y_test_cic, y_pred_test))

The Training Accuracy is:  0.8918830063287424
The Testing Accuracy is:  0.8940134009643684
              precision    recall  f1-score   support

           0       0.90      0.93      0.92     40025
           1       0.88      0.83      0.85     23851

    accuracy                           0.89     63876
   macro avg       0.89      0.88      0.89     63876
weighted avg       0.89      0.89      0.89     63876



# Modelo federado

## Definimos la función para cliente (tutorial flwr scikit-learn)

In [64]:
from typing import Tuple, Union, List
import numpy as np
from sklearn.linear_model import LogisticRegression

#  define varios tipos de datos personalizados en Python utilizando la biblioteca NumPy
XY = Tuple[np.ndarray, np.ndarray]
Dataset = Tuple[XY, XY]
LogRegParams = Union[XY, Tuple[np.ndarray]] # puede ser o un XY o un solo array
XYList = List[XY]

# Función recibe un modelo de regresión logística como entrada y devuelve los parámetros del modelo: coeficientes e intercepción
def get_model_parameters(model: LogisticRegression) -> LogRegParams:
    """Returns the paramters of a sklearn LogisticRegression model."""
    if model.fit_intercept:
        params = [
            model.coef_,
            model.intercept_,
        ]
    else:
        params = [
            model.coef_,
        ]
    return params

# Actualiza el modelo con los nuevos parámetros
def set_model_params(
    model: LogisticRegression, params: LogRegParams
) -> LogisticRegression:
    """Sets the parameters of a sklean LogisticRegression model. los coeficientes b1,b2... y la intercep b0"""
    model.coef_ = params[0]
    if model.fit_intercept:
        model.intercept_ = params[1]
    return model

# Establece los parámetros iniciales del modelo de regresión logística como ceros, necesarios ya que los parámetros del modelo no se 
# inicializan hasta que se llama a la función fit
def set_initial_params(model: LogisticRegression):
    """Sets initial parameters as zeros Required since model params are
    uninitialized until model.fit is called.

    But server asks for initial parameters from clients at launch. Refer
    to sklearn.linear_model.LogisticRegression documentation for more
    information.
    """
    n_classes = 2  
    n_features = 22  # Number of features in dataset
    model.classes_ = np.array([i for i in range(2)])

    model.coef_ = np.zeros((n_classes, n_features))
    if model.fit_intercept:
        model.intercept_ = np.zeros((n_classes,))






In [65]:
# Definimos la clase para cada cliente
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, x_train, y_train, x_test, y_test) -> None:
        self.model = model
        self.cid = cid
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

    def get_parameters(self, config):
        print(f"[Client {self.cid}] GET_PARAMETERS")
        return get_model_parameters(self.model)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] FIT")
        set_model_params(self.model, parameters)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.model.fit(self.x_train, self.y_train)
        # print(f"Training finished for round {config['server_round']}")
        return get_model_parameters(self.model), len(self.x_train), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] EVALUATE")
        set_model_params(self.model, parameters)
        loss = log_loss(self.y_test, self.model.predict_proba(self.x_test))
        accuracy = self.model.score(self.x_test, self.y_test)
        print(f"Loss: {loss}, Accuracy: {accuracy} de {self.cid}")
        return loss, len(self.x_test), {"accuracy": accuracy}

## Define model and data partition for each client

In [66]:
# Función para definir el modelo y la partición de los datasets 
# en nuestro caso 1 dataset por cliente
def client_fn(cid: str) -> fl.client.Client:
    # Creamos modelo
    # Los datos de entrada son 41 características, todos escalares
    # Create LogisticRegression Model
    model = LogisticRegression(random_state=10, solver = 'lbfgs', max_iter=300, warm_start=True)

    set_initial_params(model)

    client_datasets = [
      (X_train_cic, y_train_cic),
      (X_train_nb15, y_train_nb15),
      (X_train_toniot, y_train_toniot),
      ] 
    # Selecciona el conjunto de datos para el cliente específico
    print(f"Cliente: {int(cid)} DEFINIDO EL MODELO Y DATASET")
    full_X_train_cid, full_y_train_cid = client_datasets[int(cid)]

    # Utiliza el 10% de los datos de entrenamiento del cliente para validación
    split_idx = math.floor(len(full_X_train_cid) * 0.9)
    X_train_cid, y_train_cid = full_X_train_cid[:split_idx], full_y_train_cid[:split_idx]
    X_val_cid, y_val_cid = full_X_train_cid[split_idx:], full_y_train_cid[split_idx:]

    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Crea y devuelve el cliente
    return FlowerClient(cid, model, X_train_cid, y_train_cid, X_val_cid, y_val_cid)



    #model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    # Dataset federation (divide original data in desired clients)
    # Load data partition (divide dataset into NUM_CLIENTS distinct partitions)
      
    # partition_size = math.floor(len(X_train) / NUM_CLIENTS)
    # idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size
    # full_X_train_cid = X_train[idx_from:idx_to]
    # full_y_train_cid = y_train[idx_from:idx_to]

    # # Use 10% of the client's training data for validation
    # split_idx = math.floor(len(full_X_train_cid) * 0.9)
    # X_train_cid, y_train_cid = full_X_train_cid[:split_idx], full_y_train_cid[:split_idx]
    # X_val_cid, y_val_cid = full_X_train_cid[split_idx:], full_y_train_cid[split_idx:]

    # Create and return client
    # return FlowerClient(model, X_train_cid, y_train_cid, X_val_cid, y_val_cid)



## Define strategy and fit FL

In [37]:
def fit_round(server_round: int) -> Dict:
    """Send round number to client."""
    return {"server_round": server_round}



In [67]:
NUM_CLIENTS=3

# Create FedAvg strategy
strategy=fl.server.strategy.FedAvg(
        min_fit_clients=3,  # Never sample less than 10 clients for training
        min_evaluate_clients=3,  # Never sample less than 5 clients for evaluation
        min_available_clients=3,  # Wait until at least 75 clients are available
        # on_fit_config_fn=fit_round,
    
)

start_time = time.time()
# Start simulation of the federated learning process
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)

# Get total training time
training_time = time.time() - start_time
print("Training time: %s seconds" % (training_time))

INFO flwr 2023-04-15 12:20:26,265 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-15 12:20:30,985	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-15 12:20:32,831 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 11054810726.0, 'CPU': 40.0, 'memory': 22109621454.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 11054810726.0, 'CPU': 40.0, 'memory': 22109621454.0}
INFO flwr 2023-04-15 12:20:32,835 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-15 12:20:32,837 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
INFO flwr 2023-04-

(launch_and_get_parameters pid=226726) Cliente: 2 DEFINIDO EL MODELO Y DATASET
(launch_and_get_parameters pid=226726) [Client 2] GET_PARAMETERS
(launch_and_fit pid=226726) Cliente: 1 DEFINIDO EL MODELO Y DATASET
(launch_and_fit pid=226726) [Client 1] FIT
(launch_and_fit pid=226729) Cliente: 0 DEFINIDO EL MODELO Y DATASET
(launch_and_fit pid=226729) [Client 0] FIT
(launch_and_fit pid=226737) Cliente: 2 DEFINIDO EL MODELO Y DATASET
(launch_and_fit pid=226737) [Client 2] FIT


DEBUG flwr 2023-04-15 12:20:45,633 | server.py:229 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-15 12:20:45,641 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-15 12:20:45,645 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 3)
(launch_and_evaluate pid=226726) /usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
(launch_and_evaluate pid=226726)   warnings.warn(
(launch_and_evaluate pid=226729) /usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
(launch_and_evaluate pid=226729)   warnings.warn(
(launch_and_evaluate pid=226737) /usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X 

(launch_and_evaluate pid=226726) Cliente: 1 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226726) [Client 1] EVALUATE
(launch_and_evaluate pid=226729) Cliente: 0 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226729) [Client 0] EVALUATE
(launch_and_evaluate pid=226737) Cliente: 2 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226737) [Client 2] EVALUATE
(launch_and_evaluate pid=226726) Loss: 1.2847568220747765, Accuracy: 0.1787016776075857 de 1


(launch_and_evaluate pid=226729) /usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
(launch_and_evaluate pid=226729)   warnings.warn(
DEBUG flwr 2023-04-15 12:20:46,235 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-15 12:20:46,237 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-04-15 12:20:46,239 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 3)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 3)


...

(launch_and_evaluate pid=226726) Cliente: 2 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226726) [Client 2] EVALUATE
(launch_and_evaluate pid=226729) Cliente: 1 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226729) [Client 1] EVALUATE
(launch_and_evaluate pid=226737) Cliente: 0 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226737) [Client 0] EVALUATE


DEBUG flwr 2023-04-15 12:21:28,424 | server.py:179 | evaluate_round 4 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 4 received 3 results and 0 failures
DEBUG flwr 2023-04-15 12:21:28,426 | server.py:215 | fit_round 5: strategy sampled 3 clients (out of 3)
DEBUG:flwr:fit_round 5: strategy sampled 3 clients (out of 3)


(launch_and_evaluate pid=226726) Loss: 0.5445084229601056, Accuracy: 0.753833796909164 de 2
(launch_and_evaluate pid=226729) Loss: 1.3137304297723518, Accuracy: 0.1418151505678858 de 1
(launch_and_fit pid=226726) Cliente: 0 DEFINIDO EL MODELO Y DATASET
(launch_and_fit pid=226726) [Client 0] FIT
(launch_and_fit pid=226729) Cliente: 2 DEFINIDO EL MODELO Y DATASET
(launch_and_fit pid=226729) [Client 2] FIT
(launch_and_evaluate pid=226737) Loss: 0.580931820380035, Accuracy: 0.5516809518218465 de 0
(launch_and_fit pid=226737) Cliente: 1 DEFINIDO EL MODELO Y DATASET
(launch_and_fit pid=226737) [Client 1] FIT


DEBUG flwr 2023-04-15 12:21:40,792 | server.py:229 | fit_round 5 received 3 results and 0 failures
DEBUG:flwr:fit_round 5 received 3 results and 0 failures
DEBUG flwr 2023-04-15 12:21:40,798 | server.py:165 | evaluate_round 5: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 5: strategy sampled 3 clients (out of 3)
(launch_and_evaluate pid=226729) /usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
(launch_and_evaluate pid=226729)   warnings.warn(
(launch_and_evaluate pid=226729) /usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
(launch_and_evaluate pid=226729)   warnings.warn(
(launch_and_evaluate pid=226729) /usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
(launch_and_evaluat

(launch_and_evaluate pid=226729) Cliente: 2 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226729) [Client 2] EVALUATE
(launch_and_evaluate pid=226729) Loss: 0.5369363225886967, Accuracy: 0.755336080082105 de 2
(launch_and_evaluate pid=226729) Cliente: 0 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226729) [Client 0] EVALUATE
(launch_and_evaluate pid=226737) Cliente: 1 DEFINIDO EL MODELO Y DATASET
(launch_and_evaluate pid=226737) [Client 1] EVALUATE
(launch_and_evaluate pid=226729) Loss: 0.6120851107722797, Accuracy: 0.5314860475128175 de 0
Training time: 74.90498948097229 seconds


## Check accuracy of FL


In [42]:
def plot_metric_from_history(
    hist: None,
    save_plot_path: None,
    metric_type: None,
) -> None:
    """Function to plot from Flower server History.
    Parameters
    ----------
    hist : History
        Object containing evaluation for all rounds.
    save_plot_path : Path
        Folder to save the plot to.
    metric_type : Literal["centralized", "distributed"]
        Type of metric to plot.
    """
    metric_dict = (
        hist.metrics_centralized
        if metric_type == "centralized"
        else hist.metrics_distributed
    )
    rounds, values = zip(*metric_dict["accuracy"])
    plt.plot(np.asarray(rounds), np.asarray(values), label="FedAvg")
    # plt.savefig(Path(save_plot_path) / Path(f"{metric_type}_metrics{suffix}.png"))
    # plt.close()